# POMCP

POMCP is one of the most widely-used online POMDP methods. It is a variant of Monte Carlo tree search in which each node corresponds to an action-observation history.

(To get the required packages for this notebook, make sure to run the `install.jl` script)

In [1]:
using POMDPModels # for the crying baby problem
using POMDPModelTools
using POMDPs
using BasicPOMCP
using Interact
using Random
rng = MersenneTwister(7);

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/6137858c288834145b218ff0d1876943f939c631-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/6137858c288834145b218ff0d1876943f939c631-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [2]:
pomdp = BabyPOMDP()
b = initialstate_distribution(pomdp);

Since POMCP is an online method, it does no offline computation before being deployed into the environment. Thus the `solve` function from `POMDPs.jl` does not computation, but instead creates a planner optimized for the problem.

In [3]:
solver = POMCPSolver(c=10.0); # c is the exploration constant
planner = solve(solver, pomdp);

## Building the tree

To make a decision, POMCP constructs a tree by repeating the following process:
1. run a simulation through the tree choosing actions based on the *UCB criterion*
2. when a leaf node is reached, expand the node one level deeper and continue the simulation with a *rollout policy* (often random)
3. propagate the rewards back up the tree to maintain an estimate of the *value*\* for each action

\*This is denoted by *V* in the `BasicPOMCP` package and the original paper that describes POMCP, but it is the same thing as the state-action value typically denoted by *Q*.

In [4]:
N = 10
tree = BasicPOMCP.POMCPTree(pomdp)
trees = [deepcopy(tree)]
for i in 1:N
    s = rand(rng, b)
    BasicPOMCP.simulate(planner, s, BasicPOMCP.POMCPObsNode(tree, 1), 10)
    push!(trees, deepcopy(tree))
end

In [5]:
@manipulate for i in 1:length(trees)
    D3Tree(trees[i], init_expand=100, init_duration=0)
end

Widget{:manipulate,Any}(OrderedDict{Symbol,Any}(:i=>Widget{:slider,Int64}(OrderedDict{Symbol,Any}(:changes=>Observable{Int64}("ob_02", 0, Any[SyncCallback(Scope("knockout-component-8845884c-7b85-46dc-b6ae-35656493d8b1", Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any["i"], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")), 1)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"text-align:right;width:18%")), 2), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :input), Any[], Dict{Symbol,Any}(:max=>11,:min=>1,:attributes=>Dict{Any,Any}(:type=>"range",Symbol("data-bind")=>"numericValue: value, valueUpdate: 'input', event: {change : function () {this.changes(this.changes()+1)}}","orient"=>"horizontal"),:step=>1,:className=>"slider slider is-fullwidth",:style=>Dict{Any,Any}()), 0)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"flex-grow:1; margin: 0 2%")), 1), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"text: value")), 0)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"width:18%")), 1)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"display:flex; justify-content:center; align-items:center;")), 7), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes"=>(Observable{Int64}(#= circular reference @-6 =#), nothing),"value"=>(Observable{Int64}("ob_01", 6, Any[SyncCallback(Scope(#= circular reference @-6 =#), SyncCallback(Scope(#= circular reference @-7 =#), ##40#42{Scope,String}(Scope(#= circular reference @-8 =#), "value"))), #g#15{##16#17{##3#5,Observable{Any}},Tuple{Observable{Int64}}}(##16#17{##3#5,Observable{Any}}(##3#5(), Observable{Any}("ob_03", D3Tree(Array{Int64,1}[[7, 8], [9, 10], [11, 12], [13, 14], [15, 16], [17, 18], [2], [4, 3], [6], [], [], [5], [], [], [], [], [], []], ["o: <root>\nN: 5         ", "o: false\nN: 1         ", "o: true\nN: 1         ", "o: false\nN: 0         ", "o: false\nN: 0         ", "o: false\nN: 0         ", "a: true\nN: 2       V: -23.1     ", "a: false\nN: 3       V: -19.1     ", "a: true\nN: 1       V: -19.1     ", "a: false\nN: 0       V: 0         ", "a: true\nN: 0       V: 0         ", "a: false\nN: 1       V: -34.8     ", "a: true\nN: 0       V: 0         ", "a: false\nN: 0       V: 0         ", "a: true\nN: 0       V: 0         ", "a: false\nN: 0       V: 0         ", "a: true\nN: 0       V: 0         ", "a: false\nN: 0       V: 0         "], ["o: <root>\nN: 5\n2 children\n", "o: false\nN: 1\n2 children\n", "o: true\nN: 1\n2 children\n", "o: false\nN: 0\n2 children\n", "o: false\nN: 0\n2 children\n", "o: false\nN: 0\n2 children\n", "a: true\nN: 2\nV: -23.086382495000002\n1 children\n", "a: false\nN: 3\nV: -19.11162333\n2 children\n", "a: true\nN: 1\nV: -19.126525550000004\n1 children\n", "a: false\nN: 0\nV: 0.0\n0 children\n", "a: true\nN: 0\nV: 0.0\n0 children\n", "a: false\nN: 1\nV: -34.7816395\n1 children\n", "a: true\nN: 0\nV: 0.0\n0 children\n", "a: false\nN: 0\nV: 0.0\n0 children\n", "a: true\nN: 0\nV: 0.0\n0 children\n", "a: false\nN: 0\nV: 0.0\n0 children\n", "a: true\nN: 0\nV: 0.0\n0 children\n", "a: false\nN: 0\nV: 0.0\n0 children\n"], ["", "", "", "", "", "", "stroke:#A92B00", "stroke:#8C3A00", "stroke:#8C3A00", "stroke:#008000", "stroke:#008000", "stroke:#FF0000", "stroke:#008000", "stroke:#008000", "stroke:#008000", "stroke:#008000", "stroke:#008000", "stroke:#008000"], ["stroke-width:20.0", "stroke-width:8.94427190999916", "stroke-width:8.94427190999916", "stroke-width:1.0", "stroke-width:1.0", "stroke-width:1.0", "stroke-width:12.649110640673518", "stroke-width:15.491933384829668", "stroke-width:8.94427190999916", "stroke-width:1.0", "stroke-width:1.0", "stroke-width:8.94427190999916", "stroke-width:1.0", "stroke-width:1.0", "stroke-width:1.0", "stroke-width:1.0", "stroke-width:1.0", "stroke-width:1.0"], "POMCP Tree", Dict{Symbol,Any}(:init_expand=>100,:init_duration=>0,:tooltip=>["o: <root>\

## Online operation

When the agent is interacting with the environment, it constructs a new tree at each step, makes a decision. For this demonstation, we'll use the LaserTag problem.

In [6]:
using LaserTag
using Base64
rng = MersenneTwister(14)
laser = gen_lasertag(rng=rng)
s = initialstate(laser, rng)
sp,o,r = generate_sor(laser, s, 1, rng)
# HTML("""<div style="width: 400px; margin: 0 auto">
#     $(stringmime("image/svg+xml", LaserTagVis(laser, s=sp, o=o)))
#         </div>
#     """)

┌ Info: Recompiling stale cache file /home/shushman/.julia/compiled/v1.0/LaserTag/8wbmK.ji for LaserTag [041f53e1-e4f8-54ec-814d-e9e995aa38d4]
└ @ Base loading.jl:1184
┌ Warning: `MArray{S,T,N,L}()` is deprecated, use `MArray{S,T,N,L}(undef)` instead
│   caller = ip:0x0
└ @ Core :-1


(LTState([8, 2], [4, 2], false), [3, 1, 1, 1, 1, 0, 0, 3], -1.0)

Since the problem is discrete, we can solve the fully-observable version offline and use the resulting value estimate instead of rollouts. We use an exploration constant of 20 and limit to 1000 iterations (the visualization becomes slower if more than 1000 are used, but many more should be used to get a good solution for the problem).

In [7]:
using DiscreteValueIteration
using POMDPModelTools
using BasicPOMCP
using ParticleFilters
rng = MersenneTwister(13)
solver = POMCPSolver(tree_queries=1000,
                      c=20.0,
                      max_depth=100,
                      estimate_value=FOValue(ValueIterationSolver()),
                      rng=rng
                     )
planner = solve(solver, laser);

INFO: POMDPs.jl requirements for solve(::ValueIterationSolver, ::Union{MDP,POMDP}) and dependencies. ([✔] = implemented correctly; [X] = missing)

For solve(::ValueIterationSolver, ::Union{MDP,POMDP}):
  [✔] discount(::LaserTagPOMDP)
  [✔] n_states(::LaserTagPOMDP)
  [✔] n_actions(::LaserTagPOMDP)
  [✔] transition(::LaserTagPOMDP, ::LTState, ::Int64)
  [✔] reward(::LaserTagPOMDP, ::LTState, ::Int64, ::LTState)
  [✔] stateindex(::LaserTagPOMDP, ::LTState)
  [✔] actionindex(::LaserTagPOMDP, ::Int64)
  [✔] actions(::LaserTagPOMDP, ::LTState)
  [X] support(::LTTransDist)
  [✔] pdf(::LTTransDist, ::LTState)
For ordered_states(::Union{MDP,POMDP}) (in solve(::ValueIterationSolver, ::Union{MDP,POMDP})):
  [✔] states(::LaserTagPOMDP)
For ordered_actions(::Union{MDP,POMDP}) (in solve(::ValueIterationSolver, ::Union{MDP,POMDP})):
  [✔] actions(::LaserTagPOMDP)
Note: Missing methods are often due to incorrect importing. You must explicitly import POMDPs functions to add new methods.



MethodError: MethodError: no method matching support(::LaserTag.LTTransDist)
Closest candidates are:
  support(!Matched::BoolDistribution) at /home/shushman/.julia/packages/POMDPModelTools/eHEjm/src/distributions/bool.jl:25
  support(!Matched::POMDPModels.TMazeStateDistribution) at /home/shushman/.julia/packages/POMDPModels/IB3ea/src/TMazes.jl:55
  support(!Matched::POMDPModels.TMazeInit) at /home/shushman/.julia/packages/POMDPModels/IB3ea/src/TMazes.jl:80
  ...

At each step of the simulation, the belief is updated, the planner constructs a tree, and makes an action decision based on that tree.

In [10]:
using POMDPSimulators
ltrees = []
up = SIRParticleFilter(laser, 10000, rng=rng)

history = sim(laser, updater=up, max_steps=100, show_progress=true) do b
    a = action(planner, b) # constructs tree and makes decision
    push!(ltrees, get(planner._tree))
    return a
end;

ArgumentError: ArgumentError: Sampler for this object is not defined

The belief state and tree constructed at each step are shown below.

In [9]:
using Interact
using D3Trees
value = Interact.value
steps = eachstep(history, "a,r,sp,o,bp")
BasicPOMCP.node_tag(a::Int) = LaserTag.ACTION_NAMES[a] 
@manipulate for i in 1:length(ltrees)
    ltree = stringmime("text/html", D3Tree(ltrees[i],
                                          init_duration=0,
                                          init_expand=1,
                                          svg_height=300))
    pic = stringmime("image/svg+xml", LaserTagVis(laser, steps[i]...))
    HTML("""<div style="width: 300px; margin: 0 auto">$pic</div><div>$ltree</div>""")
end

ArgumentError: ArgumentError: Package D3Trees not found in current path:
- Run `Pkg.add("D3Trees")` to install the D3Trees package.
